In [30]:
!pip install pyspark

In [31]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
import torch
import torch.nn as nn
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [32]:
# Initialize Spark Session
spark = SparkSession.builder.appName("Food Price Prediction").getOrCreate()

# Load Data

In [33]:
path = '/content/drive/MyDrive/Distributed-Food-Price-Prediction-for-Kenyan-Markets/data/cleaned_data.csv'
data = spark.read.csv(path, header=True, inferSchema=True)

In [34]:
data.show(30)

+-------------+-----------+--------------------+---------+---------+--------------------+----------+----------------+-------+----+-----+--------------+---------------+--------+-------------------+----------+---------------------+------------+------------------+------------------+
|     province|   district|              market| latitude|longitude|           commodity|      unit|       priceflag|  price|year|month|milk and dairy|pulses and nuts|non-food|meat, fish and eggs|#item+type|vegetables and fruits|oil and fats|cereals and tubers|miscellaneous food|
+-------------+-----------+--------------------+---------+---------+--------------------+----------+----------------+-------+----+-----+--------------+---------------+--------+-------------------+----------+---------------------+------------+------------------+------------------+
|   #adm1+name| #adm2+name|    #loc+market+name| #geo+lat| #geo+lon|          #item+name|#item+unit|#item+price+flag| #value|NULL| NULL|             0|      

In [35]:
data.printSchema()

root
 |-- province: string (nullable = true)
 |-- district: string (nullable = true)
 |-- market: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- commodity: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- priceflag: string (nullable = true)
 |-- price: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- milk and dairy: integer (nullable = true)
 |-- pulses and nuts: integer (nullable = true)
 |-- non-food: integer (nullable = true)
 |-- meat, fish and eggs: integer (nullable = true)
 |-- #item+type: integer (nullable = true)
 |-- vegetables and fruits: integer (nullable = true)
 |-- oil and fats: integer (nullable = true)
 |-- cereals and tubers: integer (nullable = true)
 |-- miscellaneous food: integer (nullable = true)



In [36]:
# Feature selection
selected_features = ['year', 'month', 'commodity', 'longitude', 'latitude']
label = 'price'

In [37]:
# String indexing for categorical features
indexers = [StringIndexer(inputCol=col, outputCol=col+"_index").fit(data) for col in ['commodity']]
for indexer in indexers:
    data = indexer.transform(data)

In [38]:
from pyspark.sql.functions import col

# Converting 'latitude', 'longitude', and 'price' to double
data = data.withColumn('latitude', col('latitude').cast('double'))
data = data.withColumn('longitude', col('longitude').cast('double'))
data = data.withColumn('price', col('price').cast('double'))

# Assembling features again
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['year', 'month', 'commodity_index', 'longitude', 'latitude'], outputCol='features')

# Transform data
data = assembler.transform(data)

In [39]:
# Train-test split
train_data, test_data = data.randomSplit([0.8, 0.2])

# PySpark MLlib: Linear Regression

In [27]:
from pyspark.sql.functions import col

# Cast 'price' column to float
train_data = train_data.withColumn('price', col('price').cast('float'))
test_data = test_data.withColumn('price', col('price').cast('float'))

# Fit the Linear Regression model
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol='features', labelCol='price')

lr_model = lr.fit(train_data)

Py4JJavaError: An error occurred while calling o320.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 12.0 failed 1 times, most recent failure: Lost task 0.0 in stage 12.0 (TID 10) (f930c2b15197 executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`VectorAssembler$$Lambda$3763/0x00000008415b5840`: (struct<year_double_VectorAssembler_1ec685bebb4b:double,month_double_VectorAssembler_1ec685bebb4b:double,commodity_index:double,longitude_double_VectorAssembler_1ec685bebb4b:double,latitude_double_VectorAssembler_1ec685bebb4b:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.Ge

# Save Model

In [ ]:
rf_model.save('/content/drive/MyDrive/Distributed-Food-Price-Prediction-for-Kenyan-Markets/food_price_model')

# Git Version control

In [ ]:
%cd /content/drive/MyDrive/Distributed-Food-Price-Prediction-for-Kenyan-Markets

/content/drive/MyDrive/Distributed-Food-Price-Prediction-for-Kenyan-Markets


In [ ]:
!git pull origin main

From https://github.com/brc0d3s/Distributed-Food-Price-Prediction-for-Kenyan-Markets
 * branch            main       -> FETCH_HEAD
Already up to date.


In [ ]:
!git config --global user.email "brc0d3s@gmail.com"
!git config --global user.name "brc0d3s"

In [ ]:
!git add .

In [ ]:
!git commit -m "model"

[dev 2d182a7] model
 16 files changed, 3 insertions(+), 504 deletions(-)
 rewrite Modelling.ipynb (97%)
 create mode 100644 food_price_model/data/._SUCCESS.crc
 create mode 100644 food_price_model/data/.part-00000-f990ddec-df7f-4509-a3f4-03829edad1cc-c000.snappy.parquet.crc
 create mode 100644 food_price_model/data/_SUCCESS
 create mode 100644 food_price_model/data/part-00000-f990ddec-df7f-4509-a3f4-03829edad1cc-c000.snappy.parquet
 create mode 100644 food_price_model/metadata/._SUCCESS.crc
 create mode 100644 food_price_model/metadata/.part-00000.crc
 create mode 100644 food_price_model/metadata/_SUCCESS
 create mode 100644 food_price_model/metadata/part-00000
 create mode 100644 food_price_model/treesMetadata/._SUCCESS.crc
 create mode 100644 food_price_model/treesMetadata/.part-00000-f9498d39-f685-4c1d-9604-32c46b66f1e5-c000.snappy.parquet.crc
 create mode 100644 food_price_model/treesMetadata/_SUCCESS
 create mode 100644 food_price_model/treesMetadata/part-00000-f9498d39-f685-4c1d-

In [ ]:
!git push origin dev

Enumerating objects: 23, done.
Counting objects: 100% (23/23), done.
Delta compression using up to 2 threads
Compressing objects: 100% (15/15), done.
Writing objects: 100% (18/18), 2.94 MiB | 5.08 MiB/s, done.
Total 18 (delta 4), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (4/4), completed with 4 local objects.
To https://github.com/brc0d3s/Distributed-Food-Price-Prediction-for-Kenyan-Markets.git
   44b1be2..2d182a7  dev -> dev
